In [ ]:
%pylab inline
from scipy.stats import nbinom
from pyfastnbfit import fastfit_negbin

In [ ]:
a = 30
b = 7
p = 1 / (b + 1)
y = nbinom.rvs(n=a, p=1 / (b + 1), size=100)
fastfit_negbin(y)

In [ ]:
print(",".join(map(lambda x: str(x), y)))

In [ ]:
# test a poisson
#x4 <- rnegbin(500, mu = 10, theta = 400)
y = [6,10,8,11,8,12,9,12,13,10,10,9,7,6,10,5,8,10,9,9,10,12,3,9,6,9,8,11,6,6,11,9,12,14,12,11,6,7,13,8,7,7,13,6,5,17,7,11,7,12,6,16,8,6,10,8,8,8,11,6,8,9,10,8,11,17,10,10,10,13,13,9,10,6,5,9,5,9,13,11,8,8,8,7,10,9,14,6,11,11,8,4,11,7,5,6,13,13,8,9,10,8,12,12,4,9,11,16,11,8,9,6,10,6,16,18,7,10,12,9,10,8,4,6,6,14,9,11,11,9,10,9,12,11,7,9,7,10,8,10,12,8,14,7,14,9,3,7,10,10,11,8,5,10,12,8,10,11,13,14,9,7,13,14,8,10,18,8,14,12,10,4,19,7,5,7,12,6,8,9,8,11,13,10,12,12,8,7,11,10,9,6,11,6,6,10,10,5,8,14,9,12,11,10,8,10,8,9,10,14,12,13,5,7,16,13,11,13,14,8,9,9,17,11,12,9,10,7,9,12,16,9,15,12,5,9,9,5,16,6,8,10,10,10,9,13,16,12,11,15,16,8,7,13,13,16,8,7,9,11,10,10,9,7,7,8,13,7,7,8,10,16,5,10,11,8,7,12,18,8,13,11,12,8,11,9,8,12,7,11,8,7,12,16,6,7,12,10,9,17,10,15,11,10,8,9,12,11,10,13,10,10,11,15,11,6,10,8,10,10,6,9,17,7,13,10,14,7,12,12,6,13,9,12,5,7,10,12,8,5,11,11,12,14,11,15,6,11,14,16,6,11,10,13,7,7,2,12,11,13,9,9,11,16,8,14,6,14,7,4,10,6,11,11,9,14,11,13,10,3,8,16,17,13,8,9,7,9,15,17,8,3,5,9,12,11,8,15,10,19,13,12,12,9,14,12,12,10,10,9,12,13,8,6,9,10,10,8,8,6,12,7,11,7,6,9,12,7,9,13,7,7,11,9,6,9,10,10,7,14,11,8,14,7,9,12,6,6,15,10,13,12,2,4,9,13,11,5,11,4,5,12,12,6,8,9,9,10,6,9,9,10,6,9,6,4,9,12,12,8,7,9,12,9,6,10,9,15,14,14,9,16,12,9,11,6,10,11,7,3]
fastfit_negbin(y)